In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as torch_nn
from torch import Tensor
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import OrdinalEncoder
from tqdm.contrib.itertools import product
from tqdm.notebook import tqdm

In [2]:
NUM_SITES_EACH_PACK = 2
NUM_generaS_EACH_PACK = 4
BATCH_SIZE = 25
LR = 1e-4
EPS = 1e-6
N_EPOCHS = 50
USE_REGULARIZATIOn = True

DEVICE = "mps"

## 0. Load data

In [3]:
path = "/Users/macos/Uni/1st_year/period_3/DSProj/data/AllSites_SiteOccurrences_AllGenera_26.1.24.csv"

df = pd.read_csv(path)

df.head()

,SITE_NAME,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,Aba Zawei,1,1,1,1,0,0,0,0,0,...,4,4,0,0.0,0.000000,4,0,NaN,1.000000,0.0265
1,Abric Romani,1,0,1,0,1,1,1,1,1,...,12,12,0,0.0,0.000000,6,5,1.2,0.500000,0.0550
2,Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,...,7,5,2,0.4,0.285714,5,2,2.5,0.714286,0.2100
3,Adler cave,1,0,0,0,0,1,0,1,0,...,10,5,5,1.0,0.500000,6,4,1.5,0.600000,0.0275
4,Adyrgan,1,0,0,1,0,0,0,0,0,...,11,5,6,1.2,0.545455,11,0,NaN,1.000000,2.2000


## 1. Preprocess

### 1.1. Remove redundant columns

In [4]:
cols_redundant = ['LAT',
 'LONG',
 'ALTITUDE',
 'MAX_AGE',
 'BFA_MAX',
 'BFA_MAX_ABS',
 'MIN_AGE',
 'BFA_MIN',
 'BFA_MIN_ABS',
 'COUNTRY',
 'age_range',
 'Total_Gen_Count',
 'Large_GenCount',
 'Small_GenCount',
 'smallperlarge',
 'smallprop',
 'Herb_GenCount',
 'Nonherb_GenCount',
 'DietRatio',
 'HerbProp',
 'mid_age'
 ]

df = df.drop(columns=cols_redundant).set_index('SITE_NAME')

df.head()

,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,Homo,...,Euarctos,Paracervulus,Eostyloceros,Cervocerus,Antispiroides,Sinoryx,Prospalax,Pliopetaurista,Predicrostonyx,Boocercus
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abric Romani,1,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adler cave,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Adyrgan,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.2. Do OrdinalEncode

In [5]:
list_sites = df.index
list_generas = df.columns

In [6]:
enc_genera, enc_site = OrdinalEncoder(), OrdinalEncoder()

enc_genera.fit(df.columns.to_numpy().reshape(-1, 1))
enc_site.fit(df.index.to_numpy().reshape(-1, 1))

OrdinalEncoder()

## 2. Create data

In [7]:
def iterate_pack(values: list, n: int):
    assert len(values) % n == 0

    for i in range(0, len(values), n):
        yield values[i:i+n]


data = []
for sites, generas in product(
    iterate_pack(list_sites, NUM_SITES_EACH_PACK),
    iterate_pack(list_generas, NUM_generaS_EACH_PACK)
):
    occurence = df.loc[sites, generas].to_numpy().astype(np.float32)
    sites_encoded = enc_site.transform(sites.to_numpy().reshape(-1, 1)).squeeze().astype(np.int32)
    generas_encoded = enc_genera.transform(generas.to_numpy().reshape(-1, 1)).squeeze().astype(np.int32)

    data.append({
        'occurence': occurence,
        'sites': sites_encoded,
        'generas': generas_encoded
    })


0it [00:00, ?it/s]

In [8]:
data_train, data_val = data[:38000], data[38000:]

In [9]:
class FossilNOW(Dataset):
    def __init__(self, data: list) -> None:
        super().__init__()

        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.data[index]    
    
        occurence = torch.tensor(x['occurence'], device=DEVICE, dtype=torch.float32)
        sites = torch.tensor(x['sites'], device=DEVICE, dtype=torch.int32)
        generas = torch.tensor(x['generas'], device=DEVICE, dtype=torch.int32)

        return occurence, sites, generas
    
dataset_train, dataset_val = FossilNOW(data_train), FossilNOW(data_val)
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)
    

## 3. Model

In [10]:
class MF(torch_nn.Module):
    def __init__(
        self, 
        n_sites: int, 
        n_generas: int, 
        d_hid: int = 64,
        use_regularization: bool = True
    ) -> None:
        super().__init__()

        self.use_reg = use_regularization

        self.emd_site = torch_nn.Embedding(n_sites, d_hid)
        self.emd_genera = torch_nn.Embedding(n_generas, d_hid)

        self.criterion = torch_nn.MSELoss(reduction='none')

    def forward(self, idx_sites: Tensor, idx_generas: Tensor, occurence: Tensor):
        embedding_sites = self.emd_site(idx_sites)
        # [bz, n_sites_pack, d_hid]
        embedding_generas = self.emd_genera(idx_generas)
        # [bz, n_generas_pack, d_hid]

        embedding_generas_T = torch.permute(embedding_generas, (0, 2, 1))
        # [bz, d_hid, n_generas_pack]

        occurence_pred = torch.bmm(embedding_sites, embedding_generas_T)
        # [bz, n_sites_pack, n_generas_pack]


        loss = self.criterion(occurence, occurence_pred)
        # [bz, n_sites_pack * n_generas_pack]

        loss = torch.sum(occurence * loss) / (torch.sum(occurence) + EPS)

        if self.use_reg is True:
            loss_reg = torch.norm(self.emd_site._parameters['weight']) + torch.norm(self.emd_genera._parameters['weight'])
            loss += loss_reg

        return loss

In [11]:
mf = MF(len(list_sites), len(list_generas), use_regularization=False).to(device=DEVICE)
optimizer = AdamW(mf.parameters(), lr=LR, weight_decay=2e-5)

## 4. Training

In [12]:
def train(model, loader, optimizer):
    model.train()

    for x in tqdm(loader, total=len(dataset_train) // BATCH_SIZE):
        optimizer.zero_grad()

        occurence, idx_sites, idx_generas = x

        loss = model(idx_sites, idx_generas, occurence)

        loss.backward()
        optimizer.step()

def val(model, loader):
    model.eval()
    with torch.no_grad():
        losses = []
        for x in tqdm(loader, total=len(dataset_val) // BATCH_SIZE):
            occurence, idx_sites, idx_generas = x

            loss = model(idx_sites, idx_generas, occurence)

            losses.append(loss.item())

    print(f"Loss val: {sum(losses)/len(losses)}")


In [13]:
for n in range(N_EPOCHS):
    print(f"== Epoch: {n:02d}")
    
    train(mf, loader_train, optimizer)
    val(mf, loader_val)

== Epoch: 00


  0%|          | 0/1520 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

Loss val: 45.270032698089636
== Epoch: 01


  0%|          | 0/1520 [00:00<?, ?it/s]

...

  0%|          | 0/102 [00:00<?, ?it/s]

Loss val: 36.71207874289994
== Epoch: 49


  0%|          | 0/1520 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

Loss val: 36.58792802025971


In [15]:
emb_site = mf.emd_site._parameters['weight'].detach().cpu().numpy()
emb_species = mf.emd_genera._parameters['weight'].detach().cpu().numpy()

np.save("embd_site_mf.np", emb_site)
np.save("embd_species_mf.np", emb_species)

In [17]:
ordinal_enc_species = enc_genera.categories_
ordinal_enc_site = enc_site.categories_

np.save("ordinal_enc_site_mf.np", ordinal_enc_site)
np.save("ordinal_enc_species_mf.np", ordinal_enc_species)